## Exploring TEEHR's Additional Features

---

### TEEHR allows us to ask questions about the data and models through:
- Filtering
- Grouping
- Calculated fields
- Location attributes

### Let's go through a few examples.
Here we're using a pre-created TEEHR Evaluation dataset based on test Nextgen output.

In [ ]:
from pathlib import Path
import os
import shutil

import teehr
from utils import teehr_ngiab
from teehr.evaluation.utils import print_tree
from pyspark.sql.functions import min, max

import hvplot.pandas  # noqa

In [ ]:
MOUNTED_DATA_DIR = Path(os.environ.get("NGIAB_OUTPUT_DIR"))
configuration_name = teehr_ngiab.sanitize_string(MOUNTED_DATA_DIR.name)
print(f"NGIAB output directory: {MOUNTED_DATA_DIR}")

#### Initialize the Evaluation object

In [ ]:
TEEHR_EVALUATION_DIR = Path("/app/data")

# Initialize an Evaluation object from the directory
ev = teehr.Evaluation(dir_path=TEEHR_EVALUATION_DIR)

##### We have timeseries from 6 different sources.

In [ ]:
ev.configurations.to_pandas()

In [ ]:
ev.primary_timeseries.to_sdf().select(min("value_time"), max("value_time")).show()

In [ ]:
locations_gdf = ev.locations.to_geopandas()
print(f"Number of sites: {locations_gdf.index.size}")
locations_gdf.hvplot.points(geo=True, tiles=True).opts(width=800, height=400)

#### Q. What's the best performing model across all locations?

In [ ]:
smetrics_df = ev.metrics.query(
    group_by=["configuration_name"],
    include_metrics=[
        teehr.DeterministicMetrics.KlingGuptaEfficiency()
    ],
    order_by=["kling_gupta_efficiency"]
).to_pandas()
metrics_df

#### Location Attributes
##### Give us more variables to use in our grouping and filtering 

In [ ]:
location_attributes_gdf = ev.location_attributes.to_geopandas()
location_attributes_gdf

##### List the unique location attributes

In [ ]:
location_attributes_gdf.attribute_name.unique()

##### The location attributes have been added to the `joined_timeseries` table

In [ ]:
ev.joined_timeseries.fields()

#### Now we can make use of the location attributes in our metric calculations and questions

#### Q. How does model performance relate to stream order?

In [ ]:
location_attributes_gdf.to_crs("EPSG:4326", inplace=True)
stream_order_gdf = location_attributes_gdf[
    location_attributes_gdf.attribute_name == "stream_order"
].sort_values(by="value")

stream_order_gdf.hvplot.points(
    geo=True,
    tiles=True,
    c="value",
    size=50
).opts(width=1000, height=600)

##### Now we include stream_order in the data population group

In [ ]:
metrics_df = ev.metrics.query(
    group_by=["configuration_name", "stream_order"],
    include_metrics=[
        teehr.DeterministicMetrics.KlingGuptaEfficiency()
    ],
    order_by=["configuration_name", "kling_gupta_efficiency"]
).to_pandas()
metrics_df

In [ ]:
mean_df = metrics_df[
    [
        "configuration_name",
        "stream_order",
        "kling_gupta_efficiency"
    ]
].groupby(["configuration_name", "stream_order"]).mean().sort_values(by="stream_order", ascending=False)

mean_df.hvplot.bar(height=600, width=1000, grid=True)

#### Q. How does model performance compare for high-flows vs. low flows?

##### Add timeseries-aware calculated field for Percentile Event Detection (in-memory)

In [ ]:
events_df = (
    ev.
    joined_timeseries.
    filter("primary_location_id = 'usgs-14301000'").
    filter("configuration_name = 'nwm30_retrospective'").
    add_calculated_fields(
        teehr.TimeseriesAwareCalculatedFields.PercentileEventDetection()
    ).
    filter("event = 'true'")
).to_pandas()

events_df.head()

##### Plot high flow events (above the percentile threshold)

In [ ]:
# plot events
usgs_df = ev.primary_timeseries.filter("location_id = 'usgs-14301000'").to_pandas()
usgs_plot = usgs_df.hvplot.line(x='value_time', y='value')

event_plot = events_df.hvplot.points(x='value_time', y='primary_value', color='event_id')
(usgs_plot * event_plot).opts(width=1200, height=600, show_legend=False)

##### Now let's look at low flow events (below the percentile threshold, "non-events")

In [ ]:
non_events_df = (
    ev.
    joined_timeseries.
    filter("primary_location_id = 'usgs-14301000'").
    filter("configuration_name = 'nwm30_retrospective'").
    add_calculated_fields(
        teehr.TimeseriesAwareCalculatedFields.PercentileEventDetection()
    ).
    filter("event = 'false'")
).to_pandas()

In [ ]:
non_event_plot = non_events_df.hvplot.points(x='value_time', y='primary_value', color='gray')
(usgs_plot * non_event_plot).opts(width=1200, height=600, show_legend=False)

#### How do metrics compare for events vs. non-events?

##### For each location, calculate KGE the timeseries classified as "event"

In [ ]:
event_metrics_df = (
    ev.
    metrics.
    add_calculated_fields(
        teehr.TimeseriesAwareCalculatedFields.PercentileEventDetection()
    ).
    query(
        group_by=[
            "configuration_name",
            "primary_location_id"
        ],
        filters=[
            "event = true"
        ],
        include_metrics=[
            teehr.DeterministicMetrics.KlingGuptaEfficiency()
        ]
    )
).to_pandas()
event_metrics_df.head()

##### For each location, calculate KGE for the timeseries NOT classified as "event"

In [ ]:
non_event_metrics_df = (
    ev.
    metrics.
    add_calculated_fields(
        teehr.TimeseriesAwareCalculatedFields.PercentileEventDetection()
    ).
    query(
        group_by=[
            "configuration_name",
            "primary_location_id"
        ],
        filters=[
            "event = false",
        ],
        include_metrics=[
            teehr.DeterministicMetrics.KlingGuptaEfficiency()
        ]
    )
).to_pandas()
non_event_metrics_df.head()

##### For the purposes of the visualization, let's set any KGE values < 0 to 0.

In [ ]:
event_metrics_df["kling_gupta_efficiency"] = event_metrics_df["kling_gupta_efficiency"].clip(lower=0)
non_event_metrics_df["kling_gupta_efficiency"] = non_event_metrics_df["kling_gupta_efficiency"].clip(lower=0)

##### Let's compare Event vs. Non-Event KGE using heatmaps

In [ ]:
event_metrics_df.hvplot.heatmap(
    x="configuration_name",
    y="primary_location_id",
    C="kling_gupta_efficiency",
    cmap='YlGnBu',
    height=600,
    width=600,
    title="Event KGE"
) + \
non_event_metrics_df.hvplot.heatmap(
    x="configuration_name",
    y="primary_location_id",
    C="kling_gupta_efficiency",
    cmap='YlGnBu',
    height=600,
    width=600,
    title="Non-event KGE"
)

#### Q. How well do the models capture event peaks?

In [ ]:
metrics_df = (
    ev.metrics.add_calculated_fields(
        teehr.TimeseriesAwareCalculatedFields.PercentileEventDetection()
    ).query(
        group_by=['configuration_name', 'primary_location_id', 'water_year', 'event'],
        filters=[
            "primary_location_id = 'usgs-14301000'",
            "event = true",
        ],
        include_metrics=[
            teehr.SignatureMetrics.Maximum(
                input_field_names=['primary_value'],
                output_field_name='max_primary_value'
            ),
            teehr.SignatureMetrics.Maximum(
                input_field_names=['secondary_value'],
                output_field_name='max_secondary_value'
            )
        ]
    )
    .query(
        group_by=['configuration_name', 'primary_location_id'],
        include_metrics=[
            teehr.DeterministicMetrics.RelativeBias(
                input_field_names=['max_primary_value', 'max_secondary_value'],
                output_field_name='annual_max_relative_bias'
            )
        ],
        order_by=["annual_max_relative_bias"]
    )
).to_pandas()
metrics_df

#### Q. What's the uncertainty in the metric results?

In [ ]:
boot = teehr.Bootstrappers.CircularBlock(
    seed=40,
    block_size=100,
    quantiles=[0.05, 0.5, 0.95],
    reps=100
)
kge = teehr.DeterministicMetrics.KlingGuptaEfficiency()
kge.bootstrap = boot
kge.unpack_results = True

In [ ]:
metrics_df = ev.metrics.query(
    group_by=["configuration_name"],
    include_metrics=[
        kge
    ],
    order_by=["configuration_name"]
).to_pandas()
metrics_df

#### Q. What are some ways to access and share TEEHR Evaluations?
##### Reading and cloning from cloud storage (S3)

In [ ]:
# Define the directory where the Evaluation will be created
test_eval_dir = Path(TEEHR_EVALUATION_DIR.parent, "clone_from_s3_example")
shutil.rmtree(test_eval_dir, ignore_errors=True)

# Create an Evaluation object and create the directory
ev = teehr.Evaluation(dir_path=test_eval_dir, create_dir=True)

In [ ]:
# List the evaluations in the S3 bucket
ev.list_s3_evaluations()

In [ ]:
# Clone the e0_2_location_example evaluation from the S3 bucket
ev.clone_from_s3("e0_2_location_example")

In [ ]:
locations_gdf = ev.locations.to_geopandas()
locations_gdf.hvplot.points(geo=True, tiles=True).opts(width=800, height=400)

In [ ]:
ev. primary_timeseries.to_pandas().head()

##### Reading directly from S3

In [ ]:
# Create an Evaluation object that points to the S3 location
ev = teehr.Evaluation("s3a://ciroh-rti-public-data/teehr-data-warehouse/v0_4_evaluations/e0_2_location_example")

In [ ]:
ev.locations.to_geopandas().head()